#### Import

In [ ]:
import os
import sys
import torch
import numpy as np
from tqdm import tqdm
from scipy import ndimage
from cellpose import models
from skimage import measure
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from cellpose.plot import image_to_rgb, mask_overlay

from scipy.ndimage import center_of_mass

In [ ]:
sys.path.append('..')

In [ ]:
from src.utils.image_utils import load_czi_images


#### Function

In [ ]:
# Quick version - simple 360 degree rotation
def quick_rotation_gif(masks,max_labels=15, z_scale=3.0, cell_slice_th=5, filename='quick_rotation.gif'):
    """Simple 360 degree rotation GIF"""
    
    # Plot setup
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Get and plot cells (simplified)
    unique_labels = np.unique(masks)[1:max_labels]  # First 5 cells only
    colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))
    
    def animate(frame):
        ax.clear()
        angle = frame * 10  # 10 degrees per frame
        
        for i, label in enumerate(unique_labels):
            cell_mask = (masks == label).astype(np.uint8)
            if np.sum(cell_mask) < cell_slice_th:
                continue
            try:
                # Apply smoothing to make more spherical
                cell_mask_smooth = ndimage.binary_fill_holes(cell_mask)
                cell_mask_smooth = ndimage.gaussian_filter(cell_mask_smooth.astype(float), sigma=1.0)
            
                # Use adjusted spacing to compress Z-axis
                verts, faces, normals, values = measure.marching_cubes(
                                cell_mask_smooth, 
                                level=0.5, 
                                spacing=(1, 1, 1/z_scale))
                # Plot the surface
                ax.plot_trisurf(verts[:, 2], verts[:, 1], verts[:, 0], 
                          triangles=faces, alpha=0.7, 
                          color=colors[i], label=f'Cell {label}')
            except:
                pass
        
        ax.set_title(f'Rotating 3D Cells (angle: {angle}°)')
        ax.view_init(elev=30, azim=angle)
        ax.set_xlim(0, masks.shape[2])
        ax.set_ylim(0, masks.shape[1])
        ax.set_zlim(0, masks.shape[0])
        ax.set_zlabel('Z')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
    
    anim = animation.FuncAnimation(fig, animate, frames=36, interval=200)  # 36 frames = 360°
    anim.save(filename, writer=PillowWriter(fps=5))
    print(f"Quick rotation saved as {filename}")
    plt.close()

In [ ]:
def create_gif_from_arrays(arrays, output_path='animation.gif', fps=5, cmap='gray', titles=None):
    """
    Create a GIF from a series of 2D or 3D arrays
    
    Parameters:
    - arrays: list of numpy arrays or single 3D array where first dimension is frames
    - output_path: path to save the GIF
    - fps: frames per second
    - cmap: colormap for visualization
    - titles: optional list of titles for each frame
    """
    # Handle input format
    if isinstance(arrays, np.ndarray):
        if arrays.ndim == 3:
            frame_arrays = [arrays[i] for i in range(arrays.shape[0])]
        else:
            frame_arrays = [arrays]
    else:
        frame_arrays = arrays
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Initialize with first frame
    im = ax.imshow(frame_arrays[0], cmap=cmap, animated=True)
    ax.axis('off')
    
    def animate(frame):
        im.set_array(frame_arrays[frame])
        if titles:
            ax.set_title(titles[frame])
        else:
            ax.set_title(f'Frame {frame + 1}/{len(frame_arrays)}')
        return [im]
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, frames=len(frame_arrays), 
                                 interval=1000//fps, blit=True, repeat=True)
    
    # Save as GIF
    anim.save(output_path, writer=PillowWriter(fps=fps))
    print(f"GIF saved to {output_path}")
    plt.close()

In [ ]:
def enhance_cell_image_contrast(image):
    if image.shape[0] < 4:
        image = np.transpose(image, (1, 2, 0))
    if image.shape[-1] < 3 or image.ndim < 3:
        image = image_to_rgb(image, channels=[0, 0])
    else:
        if image.max() <= 50.0:
            image = np.uint8(np.clip(image, 0, 1) * 255)
    return image

In [ ]:
def get_cell_centers(masks_array):
    labels = np.unique(masks_array)
    centers = {}
    for label in labels:
        if label == 0:
            continue
        mask = (masks_array == label).astype(np.uint8)
        z_coords, y_coords, x_coords = np.where(mask == 1)
        if len(x_coords) > 0:
            center_of_mass_x = np.mean(x_coords).astype(float)
            center_of_mass_y = np.mean(y_coords).astype(float)
            center_of_mass_z = np.mean(z_coords).astype(float)
            centers[int(label)] = (center_of_mass_x, center_of_mass_y, center_of_mass_z)
    return centers

In [ ]:
def get_cell_centers_fast(masks_array):
    """
    Ultra-fast computation of cell centers using scipy.ndimage.center_of_mass.
    
    Parameters:
    - masks_array: 3D numpy array with labeled cells
    
    Returns:
    - centers: dict mapping cell labels to (x, y, z) center coordinates
    """
    
    
    # Get all unique labels (excluding background)
    labels = np.unique(masks_array)
    labels = labels[labels > 0]
    
    if len(labels) == 0:
        return {}
    
    # Compute centers of mass for all labels at once
    centers_of_mass = center_of_mass(masks_array > 0, masks_array, labels)
    
    # Convert to dictionary with proper coordinate order (x, y, z)
    centers = {}
    for i, label in enumerate(labels):
        if not np.isnan(centers_of_mass[i]).any():
            # Note: center_of_mass returns (z, y, x), so we need to reorder
            z, y, x = centers_of_mass[i]
            centers[int(label)] = (float(x), float(y), float(z))
    
    return centers

#### Input

In [ ]:
czi_file_path = '/home/dafei/data/MS2/gRNA2_12.03.25-st-13-II---.czi'
output_dir = '/home/dafei/output/MS2/3d_cell_segmentation'

In [ ]:
os.makedirs(output_dir, exist_ok=True)

In [ ]:
image_data = load_czi_images(czi_file_path)

In [ ]:
# get Z stack at time t
t = 0
z_stack_t = image_data[0, t, 1, :, :, :, 0]

In [ ]:
fig, ax = plt.subplots(1, z_stack_t.shape[0], figsize=(20, 10))
for i in range(z_stack_t.shape[0]):
    img = enhance_cell_image_contrast(z_stack_t[i])
    ax[i].imshow(img, cmap='gray')
    ax[i].axis('off')
    ax[i].set_title(f'Slice {i+1}')
plt.tight_layout()
plt.show()

In [ ]:
device = torch.device('cuda:1')
model = models.CellposeModel(gpu=True, device=device)

In [ ]:
masks, flows, _ = model.eval(z_stack_t, z_axis=0, channel_axis=1,
                                batch_size=32,
                                do_3D=True, flow3D_smooth=1)

In [ ]:
z_stack_t_plus_1 = image_data[0, t+1, 1, :, :, :, 0]

In [ ]:
masks1, flows1, _ = model.eval(z_stack_t_plus_1, z_axis=0, channel_axis=1,
                                batch_size=32,
                                do_3D=True, flow3D_smooth=1, progress=tqdm)

In [ ]:
# flows  explaination
# flows[0] circular coordinates DP
# flows[1] DP in xy coordinates
# flows[2] cell probabilities

##### Flows analysis

In [ ]:
len(flows)

In [ ]:
flows[0].shape, flows[1].shape, flows[2].shape

In [ ]:
# gradient flow in slice 0 in xy coordinates
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(flows[1][0][0])
ax[0].set_title('Flow in Z')
ax[1].imshow(flows[1][1][0])
ax[1].set_title('Flow in X')
ax[2].imshow(flows[1][2][0])
ax[2].set_title('Flow in Y')
plt.show()

In [ ]:
# gradient flow in slice 0 in xy circular coordinates
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(flows[0][0][:,:,0])
ax[0].set_title('Flow in Z')
ax[1].imshow(flows[0][0][:,:,1])
ax[1].set_title('Flow in X')
ax[2].imshow(flows[0][0][:,:,2])
ax[2].set_title('Flow in Y')
plt.show()

In [ ]:
masks_array = np.array(masks)

In [ ]:
centers = get_cell_centers(masks_array)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for label, center in centers.items():
    ax.scatter(center[0], center[1], center[2], label=f'Cell {label}', s=10)
ax.set_title('Cell Centers in 3D')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(0, masks_array.shape[2])
ax.set_ylim(0, masks_array.shape[1])
ax.set_zlim(0, masks_array.shape[0])
plt.show()

In [ ]:
save_dir = os.path.join(output_dir, os.path.basename(czi_file_path).replace('.czi', ''))
os.makedirs(save_dir, exist_ok=True)

In [ ]:
save_path = os.path.join(save_dir, 'z_stack_t0_seg_masks.npy')
np.save(save_path, masks)
print(f"Segmentation results saved to {save_path}")

In [ ]:
label =  np.random.randint(1, np.max(masks))
frames = []
for i in range(masks.shape[0]):
    img = z_stack_t[i]
    img = enhance_cell_image_contrast(img)
    maski = masks[i]
    cell_mask = (maski==1).astype(np.uint8)
    overlay = mask_overlay(img, cell_mask)
    frames.append(overlay)
    

In [ ]:
create_gif_from_arrays(frames, output_path=os.path.join(save_dir, 'cell_rotation2.gif'),fps=1,
                      titles=[f'Cell {label} - Slice {i+1}' for i in range(len(frames))],
                      cmap='gray')